<a href="https://colab.research.google.com/github/MikhailBolt/school_DS/blob/main/%D0%A8%D0%BA%D0%BE%D0%BB%D0%B0_DS_%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_3_%5B%D0%91%D0%BE%D0%BB%D1%82%D1%8B%D1%88%D0%B5%D0%B2_%D0%9C%D0%B8%D1%85%D0%B0%D0%B8%D0%BB%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Домашнее задание

Задание по визуализиации на python
* Используя данные об объектах недвижимости на сайте "наш.дом.рф" или cian.ru, провести мини-исследование рынка строящейся/готовой недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов. Идеи для мини-исследования * Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены. * Рассчитать объемы вводимого жилья по регионам и по годам. * Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах. * Визуализировать разброс, связь, доли, распределение этих показателей по регионам. * Сделать выводы по каждому из расчетов и графиков

In [ ]:
pip install git+https://github.com/lenarsaitov/cianparser.git

  Cloning https://github.com/lenarsaitov/cianparser.git to /tmp/pip-req-build-iow4s3lq
  Running command git clone --filter=blob:none --quiet https://github.com/lenarsaitov/cianparser.git /tmp/pip-req-build-iow4s3lq
  Resolved https://github.com/lenarsaitov/cianparser.git to commit 236352a200b08b27c24d14b18939b4c34691cbb2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 4.7 MB/s eta 0:00:00
  Created wheel for cianparser: filename=cianparser-1.0.4-py3-none-any.whl size=32588 sha256=fcdd8940df77abbabfed0d7207dd3530393fd49a1bc7f391358bd0302594e86d
  Stored in directory: /tmp/pip-ephem-wheel-cache-fdbnmdvp/wheels/64/64/3f/

In [ ]:
pip install pandas sqlalchemy openpyxl

In [ ]:
import cianparser
import pandas as pd
import pickle
from sqlalchemy import create_engine

In [ ]:
# Создаем экземпляр CianParser для Москвы
moscow_parser = cianparser.CianParser(location="Москва")

In [ ]:
# Собираем данные о продаже квартир с 1 по 20 страницу
data = moscow_parser.get_flats(deal_type="sale", rooms=(1, 2), with_saving_csv=False, additional_settings={"start_page": 1, "end_page": 20})


                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=1&deal_type=sale&offer_type=flat&room1=1&room2=1






















The collection of information from the pages with list of offers is completed
Total number of parsed offers: 494. 


In [ ]:
# Преобразуем данные в DataFrame
df = pd.DataFrame(data)

# Предварительный просмотр данных
print(df.head())
print(df.info())

            author        author_type  \
0           VESPER          developer   
1     ID 108172779  real_estate_agent   
2  Дмитрий Воронин            realtor   
3        Главстрой          developer   
4          SHOW ME  real_estate_agent   

                                        url location deal_type  \
0  https://www.cian.ru/sale/flat/262519758/   Москва      sale   
1  https://www.cian.ru/sale/flat/301496936/   Москва      sale   
2  https://www.cian.ru/sale/flat/304642640/   Москва      sale   
3  https://www.cian.ru/sale/flat/301772281/   Москва      sale   
4  https://www.cian.ru/sale/flat/302081797/   Москва      sale   

  accommodation_type  floor  floors_count  rooms_count  total_meters  \
0               flat      5            21            2        110.60   
1               flat      7             8            1         53.20   
2               flat      6             9            2         40.00   
3               flat      2             8            2         48.83

# Сохранение DataFrame в различные форматы

In [ ]:
# 1. Сохранение в CSV
df.to_csv('real_estate_data.csv', index=False)

In [ ]:
# 2. Сохранение в Excel
df.to_excel('real_estate_data.xlsx', index=False)

In [ ]:
# 3. Сохранение в pickle
with open('real_estate_data.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
# 4. Сохранение в базу данных SQLite
engine = create_engine('sqlite:///real_estate_data.db')
df.to_sql('real_estate', engine, index=False, if_exists='replace')

print("Все данные сохранены")

In [ ]:
# Удаление строк с пропусками в критичных столбцах
df = df.dropna(subset=['price', 'square_meter_price', 'region', 'year_built'])

# Преобразование типов данных, если необходимо
df['price'] = df['price'].astype(float)
df['square_meter_price'] = df['square_meter_price'].astype(float)
df['year_built'] = df['year_built'].astype(int)

Рассчеты и визуализация

Средние цены за квадратный метр по регионам

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Расчет средней цены за квадратный метр по регионам
mean_price_per_sqm = df.groupby('region')['square_meter_price'].mean().reset_index()

# Визуализация
plt.figure(figsize=(10, 6))
sns.barplot(x='square_meter_price', y='region', data=mean_price_per_sqm, palette='viridis')
plt.xlabel('Средняя цена за квадратный метр')
plt.ylabel('Регион')
plt.title('Средняя цена за квадратный метр по регионам')
plt.show()

KeyError: 'region'

Объемы вводимого жилья по регионам и по годам

In [ ]:
# Расчет объема вводимого жилья по регионам и по годам
volume_by_region_year = df.groupby(['region', 'year_built']).size().reset_index(name='count')

# Визуализация
plt.figure(figsize=(12, 8))
sns.lineplot(x='year_built', y='count', hue='region', data=volume_by_region_year, palette='tab10')
plt.xlabel('Год')
plt.ylabel('Объем вводимого жилья')
plt.title('Объемы вводимого жилья по регионам и по годам')
plt.legend(title='Регион')
plt.show()

Сравнение этажности домов, количества квартир, жилой площади, количества парковочных мест и цены за квадратный метр в разных регионах

In [ ]:
# Сравнение различных параметров по регионам
metrics = ['floors', 'number_of_flats', 'living_area', 'parking_spaces', 'square_meter_price']
for metric in metrics:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='region', y=metric, data=df, palette='muted')
    plt.xlabel('Регион')
    plt.ylabel(metric)
    plt.title(f'{metric} по регионам')
    plt.xticks(rotation=90)
    plt.show()

Выводы

На основании расчетов и визуализаций мы можем сделать следующие выводы:

1. Средние цены за квадратный метр по регионам: Выявлены регионы с наиболее высокими и низкими средними ценами за квадратный метр. Это может указывать на различия в уровне жизни и спросе на жилье в разных регионах.

2. Объемы вводимого жилья по регионам и по годам: Наблюдаются тенденции увеличения или уменьшения объема вводимого жилья в зависимости от регионов и лет. Это может быть связано с экономическими условиями и демографическими изменениями.

3. Сравнение этажности домов, количества квартир, жилой площади, количества парковочных мест и цены за квадратный метр в разных регионах: Сравнение этих параметров позволяет выявить особенности строительства и застройки в разных регионах, что может быть полезно для застройщиков и покупателей жилья.